# Jupyter Example
## LAMMPS: pour/in.pour.2d
Reference: https://github.com/lammps/lammps/blob/master/examples/pour/in.pour.2d  
Required files: None  
Description: Pour 2d granular particles into container

In [1]:
from __future__ import print_function
import random, math
from lammps import IPyLammps

In [2]:
L = IPyLammps()
L.dimension("2")
L.atom_style("sphere")
L.boundary("f fm p")
L.newton("off")
L.comm_modify("vel yes")
L.region("reg block 0 100 0 50 -0.5 0.5 units box")
L.create_box("1 reg")
L.neighbor("0.2 bin")
L.neigh_modify("delay 0")

LAMMPS output is captured by PyLammps wrapper


In [3]:
# IMPORTANT NOTE: these values of stiffness (4000) and timestep (0.001)
# are used in this example file to produce a quick simulation and movie.
# More appropriate values for realistic simulations would be
# k = 4.0e5 and dt = 0.0001, as in bench/in.chute (if it were Hertzian).

L.pair_style("gran/hertz/history 4000.0 NULL 100.0 NULL 0.5 0")
L.pair_coeff("* *")

In [4]:
L.fix("1 all nve/sphere")
L.fix("2 all gravity 1.0 spherical 0.0 -180.0")
L.fix("xwalls all wall/gran hertz/history 4000.0 NULL 100.0 NULL 0 1 xplane 0 100")
L.fix("ywalls all wall/gran hertz/history 4000.0 NULL 100.0 NULL 0 1 yplane 0 NULL")
L.region("slab block 1.0 99.0 30 34.5 -0.5 0.5 units box")
L.fix("ins all pour 1000 1 4767548 vol 0.4 10 diam range 0.5 1.0 region slab")
L.fix("3 all enforce2d")
L.compute("1 all erotate/sphere")

In [5]:
L.dump("id all atom 250 dump.pour")

#L.dump("2 all image 500 image.*.jpg type type zoom 1.6 adiam 1.5")
#L.dump_modify("2 pad 5")

L.dump("3 all movie 500 mov.mp4 type type zoom 1.6 adiam 1.5")
L.dump_modify("3 pad 5")

In [6]:
L.thermo_style("custom step atoms ke c_1 vol")
L.thermo("1000")
L.thermo_modify("lost ignore norm no")
L.compute_modify("thermo_temp dynamic/dof yes")
L.timestep("0.001")
L.run("25000")

['Neighbor list info ...',
 '  update every 1 steps, delay 0 steps, check yes',
 '  max neighbors/atom: 2000, page size: 100000',
 '  master list distance cutoff = 1.2',
 '  ghost atom cutoff = 1.2',
 '  binsize = 0.6, bins = 167 84 2',
 '  1 neighbor lists, perpetual/occasional/extra = 1 0 0',
 '  (1) pair gran/hertz/history, perpetual',
 '      attributes: half, newton off, size, history',
 '      pair build: half/size/bin/newtoff',
 '      stencil: half/bin/2d/newtoff',
 '      bin: standard',
 'Setting up Verlet run ...',
 '  Unit style    : lj',
 '  Current step  : 0',
 '  Time step     : 0.001',
 'Per MPI rank memory allocation (min/avg/max) = 5.804 | 5.804 | 5.804 Mbytes',
 'Step Atoms KinEng c_1 Volume ',
 '       0        0           -0            0         5000 ',
 'WARNING: Less insertions than requested (../fix_pour.cpp:679)',
 '    1000      285     244.9894            0         5000 ',
 'WARNING: Less insertions than requested (../fix_pour.cpp:679)',
 'WARNING: Less inser

In [7]:
L.undump("3")
L.video("mov.mp4")